In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
import tkinter as tk
from tkinter import messagebox

def notify():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    messagebox.showinfo("Done", "Your Python script has finished running!")
    root.destroy()



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader
import matplotlib.pyplot as plt

from datetime import datetime
import time
import json

import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.transforms as transforms
import Double_Pendulum.dynamics as dynamics

import training_data as training_data
import Plotting.theta_visualiser as theta_visualizer
import learning_plotters as lp

import autoencoders

from functools import partial

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
rp = robot_parameters.LUMPED_PARAMETERS.copy()
rp["m0"] = 0.0
print(rp)

train_clockwise = False

In [ ]:
def mask_points(q0_split, clockwise = False):

	"""
	Returns a set of [q0, q1] points based on "q0_split" limits on q0 and q1.
	The limits on q1 depend on whether a clockwise or counterclockwise dataset is selected.
	"""   

	# Retrieve training points
	points = training_data.points.to(device)
	
	# Mask to retrieve only the counterclockwise points
	width_mask = (points[:,0] >= q0_split[0]) & (points[:,0] <= q0_split[1])
	ccw_mask = ((points[:,1] >= points[:,0]) & 
				  (points[:,1] <= points[:,0] + torch.pi))
	
	# Mask to retrieve only the clockwise points
	cw_mask = ((points[:,1] >= points[:,0] - torch.pi) & (points[:,1] <= points[:,0]))

	if clockwise:
		final_mask = width_mask & cw_mask
	else:
		final_mask = width_mask & ccw_mask
	
	points = points[final_mask]
	points = points[0:6000]

	if points.size(0) < 6000:
		print("Warning: Only", points.size(0), "points in dataset.")

	return(points)

In [ ]:
masked_points = mask_points((-torch.pi, torch.pi), clockwise = train_clockwise)
deshifted_points = transforms.wrap_to_pi(masked_points.clone())
lp.points_plotter(masked_points, extend="ccw")
lp.points_plotter(deshifted_points, extend="ccw")

In [ ]:
def make_dataset(points):

	"""
	Compute mass- and input matrix of all training points to reduce load in training.
	Returns TensorDataset of (q, M_q, A_q). 
	"""

	data_pairs = []
	for point in points:
		Mq_point, _, _ = dynamics.dynamical_matrices(rp, point, point)
		Aq_point = dynamics.input_matrix(rp, point)
		data_pairs.append((point, Mq_point, Aq_point))

	points_tensor = torch.stack([pair[0] for pair in data_pairs])           # Tensor of all points
	mass_matrices_tensor = torch.stack([pair[1] for pair in data_pairs])   # Tensor of all mass matrices
	input_matrices_tensor = torch.stack([pair[2] for pair in data_pairs])  # Tensor of all input matrices

	# Create TensorDataset
	dataset = TensorDataset(points_tensor, mass_matrices_tensor, input_matrices_tensor)
	return(dataset)


In [ ]:
def make_dataloaders(dataset, batch_size = 512, train_part = 0.7):

	"""
	Creates a training and validation dataloader from an input dataset, based on 
	batch size and the ratio train_part. 
	"""

	train_size = int(train_part * len(dataset))
	val_size = len(dataset) - train_size

	# Create TensorDataset for both training and testing sets
	train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

	# Create the DataLoader for both training and testing sets
	train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
	val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

	return(train_dataloader, val_dataloader)

In [ ]:
import torch.nn.functional as F





def loss_fun(q, theta, q_hat, M_q, A_q, J_h_enc, J_h_dec, loss_weights):

	"""
	Loss function for training the Autoencoder. Loss terms are the following:
	l_recon:    Loss between input- and reconstructed variable. (MSE)
	l_off_dia:  Loss of off-diagonal terms of mass matrix in theta-space. (MSE)
	l_dia:      Loss on diagonal terms of mass matrix in theta-space. 
				(mean of normalized negative log-loss)
	l_input:    Loss to drive input matrix in theta-space to [1, 0]^T (MSE)

	"""


	lw = loss_weights
	

	l_recon = F.mse_loss(q, q_hat, reduction="mean")

	# Calculate forward and inverse Jacobians
	J_h = J_h_enc
	J_h_trans = torch.transpose(J_h, 1, 2)
	J_h_inv = J_h_dec
	J_h_inv_trans = torch.transpose(J_h_inv, 1, 2)

	M_th = J_h_inv_trans @ M_q @ J_h_inv
	A_th = J_h_inv_trans @ A_q

	# Loss inspired by Pietro Pustina's paper on input decoupling:
	# https://arxiv.org/pdf/2306.07258
	l_input_jac = F.mse_loss(J_h[:, 0, :], A_q[:, :, 0], reduction="mean")

	l_J_eye = F.mse_loss( torch.bmm(J_h, J_h_inv), 
						torch.eye(2,device=J_h.device).unsqueeze(0).repeat(J_h.size(0),1,1),
						reduction='mean' )

	# Loss on the first coordinate theta, again from Pietro Pustina's analytic formulation
	theta_ana = torch.vmap(transforms.analytic_theta, in_dims = (None, 0))(rp, q)
	l_theta = F.mse_loss(theta[:, 0], theta_ana[:, 0], reduction="mean")

	# Enforce inertial decoupling
	l_off_diag = torch.mean((M_th[:, 0, 1])**2)
	diag_values = torch.diagonal(M_th, dim1=1, dim2=2)
	l_diag = torch.mean((-1 + torch.exp(-(diag_values - 1))) * (diag_values < 1).float())  # Shape: (batch_size, 2)

	## input decoupling loss
	#l_input = torch.mean((A_th[:, 1]**2)) + torch.mean(((A_th[:, 0]-1)**2))
	l_input = torch.mean((A_th[:, 1]**2))
	
	
	loss_terms = torch.tensor([l_recon * lw["w_recon"], l_diag * lw["w_diag"], l_off_diag * lw["w_off_diag"], 
							   l_input * lw["w_input"], l_input_jac * lw["w_input_jac"], l_theta * lw["w_theta"], l_J_eye * lw["w_J_eye"]])
	loss_sum = l_recon * lw["w_recon"] + l_diag * lw["w_diag"] + l_off_diag * lw["w_off_diag"] + l_input * lw["w_input"] + \
			   l_input_jac * lw["w_input_jac"] + l_theta * lw["w_theta"] + l_J_eye * lw["w_J_eye"]

	return loss_sum, loss_terms

In [ ]:

#model.load_state_dict(torch.load(load_path, weights_only=True))

def load_model(current_time, num_epochs, lr, dir_name = None):
	if dir_name is None:
		model = autoencoders.Autoencoder_double(rp).to(device)  # Move model to GPU
		dir_name = f"NN_{current_time}"
		file_counter = 0
	else:
		dir_path = os.path.join(os.getcwd(), "Models", dir_name)
		nn_filename = dir_name + "_0.pth"
		nn_filepath = os.path.join(dir_path, nn_filename)
		file_counter = 0
		while os.path.isfile(nn_filepath):
			print("Looking for latest weights and bias file.")
			file_counter += 1
			nn_filename = nn_filename[:-6] + "_" + str(file_counter) + ".pth"
			nn_filepath = os.path.join(dir_path, nn_filename)

		model_path = os.path.join("Models", dir_name, dir_name + "_" + str(file_counter-1) + ".pth")
		model = autoencoders.Autoencoder_double(rp).to(device)  # Initialize model architecture
		model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))  # Load weights

	return model, file_counter, dir_name


In [ ]:
def make_metadata(q0_split, lr, num_epochs, nn_filename, model, file_counter, dir_path, loss_weights):
	
	metadata = {"q0_low" : q0_split[0],
				"q0_high" : q0_split[1],
				"lr" : lr,
				"epochs" : num_epochs,
				"file_name" : nn_filename,
				"loss_weights": loss_weights
				}
	metadata["architecture"] = {
		"encoder": autoencoders.summarize_sequential(model.enc),
		"decoder": autoencoders.summarize_sequential(model.dec)
	}

	metadata_filename = "metadata_" + str(file_counter) + ".json"
	metadata_filepath = os.path.join(dir_path, metadata_filename)
	with open(metadata_filepath, "w") as f:
		json.dump(metadata, f, indent=4)


In [ ]:
def train_AE_model(rp, device, lr, num_epochs, q0_split, train_dataloader, val_dataloader, current_time, 
				   loss_weights, load_dir_name = None, save_dir_name = None):

	"""
	Executes training loop for autoencoder
	"""

	model, file_counter, dir_name = load_model(current_time, num_epochs, lr, dir_name)

	optimizer = torch.optim.Adam(model.parameters(), lr=lr)#,  weight_decay=1e-6)
	scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5 ** (1 / num_epochs))
	 
	dir_path = os.path.join(os.getcwd(), "Models", dir_name)
	os.makedirs(dir_path, exist_ok=True)
	nn_filename = dir_name + "_" + str(file_counter) + ".pth"
	nn_filepath = os.path.join(dir_path, nn_filename)



	make_metadata(q0_split, lr, num_epochs, nn_filename, model, file_counter, dir_path, loss_weights)

	train_losses = []
	val_losses = []
	start_time = time.time()  

	for epoch in range(num_epochs):

		# Training phase
		model.train()
		train_loss = 0
		train_loss_terms = torch.zeros(7)
		for index, (q, M_q, A_q) in enumerate(train_dataloader):
			q = q.to(device)
			M_q = M_q.to(device)
			A_q = A_q.to(device)

			batch_size = q.size(0)
			
			theta, J_h, q_hat, J_h_dec, J_h_ana = model.forward(q)  
			theta_ana = model.theta_ana(q)
					
			loss, loss_terms = loss_fun(q, theta, q_hat, M_q, A_q, J_h, J_h_dec, loss_weights)
			
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			
			train_loss += loss.item() * batch_size
			train_loss_terms += loss_terms * batch_size
		train_loss /= len(train_dataloader.dataset) 
		train_loss_terms /= len(train_dataloader.dataset) 
		train_losses.append(train_loss)

		# Validation phase
		model.eval()
		val_loss = 0
		val_loss_terms = torch.zeros(7)
		with torch.no_grad():
			for index, (q, M_q, A_q) in enumerate(val_dataloader):
				q = q.to(device)
				M_q = M_q.to(device)
				A_q = A_q.to(device)

				batch_size = q.size(0)

				theta, J_h, q_hat, J_h_dec, J_h_ana = model.forward(q)
				theta_ana = model.theta_ana(q)

				loss, loss_terms = loss_fun(q, theta, q_hat, M_q, A_q, J_h, J_h_dec, loss_weights)

				val_loss += loss.item() * batch_size
				val_loss_terms += loss_terms * batch_size
		val_loss /= len(val_dataloader.dataset)
		val_loss_terms /= len(val_dataloader.dataset)
		val_losses.append(val_loss)
		epoch_duration = time.time() - start_time
		scheduler.step()


		
		tlt = train_loss_terms
		print(
			f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Duration: {epoch_duration:.2f} seconds')
		print(
			f'l_recon: {tlt[0]:.4f}, l_diag: {tlt[1]:.4f}, l_off_diag: {tlt[2]:.4f}, l_input: {tlt[3]:.4f}, l_input_jac: {tlt[4]:.4f}, l_theta: {tlt[5]:.4f}, l_J_eye: {tlt[6]:.4f}'
		)
		#print(
		#	f'l_recon: {tlt[0]:.4f}, l_inertia: {tlt[1]:.4f}, l_input: {tlt[2]:.4f}, l_input_jac: {tlt[3]:.4f}, l_theta: {tlt[4]:.4f}'
		#)
		
	return(model, train_losses, val_losses, dir_path, nn_filepath, file_counter)


In [ ]:
def save_model(model, file_path):
	torch.save(model.state_dict(), file_path)
	print(f"Model parameters saved to {file_path}")


In [ ]:
def plot_loss(train_losses, val_losses, log = False, save = False, file_path = None):

	"""
	Plots training and validation loss. 
	ylim" and "yscale" should be enabled depending on the loss function.
	"""

	plt.figure(figsize=(10, 6))
	plt.plot(train_losses, label="Training Loss")
	plt.plot(val_losses, label="Validation Loss")
	plt.xlabel("Epoch")
	plt.ylabel("Loss")
	#plt.ylim((-1, 40))
	plt.legend()
	plt.title("Training and Validation Loss over Epochs")
	plt.grid(True)
	if log:
		plt.yscale("log")
	if save:
		plt.savefig(file_path, dpi=800)
	plt.show()


In [ ]:
def save_loss(train_losses, val_losses, save_loss_path):
	losses_dict = {
		"train": train_losses,
		"val": val_losses
	}
	
	torch.save(losses_dict, save_loss_path)
	print("Saved losses to {save_loss_path}")

In [ ]:
def load_loss(load_loss_path):
    losses_dict = torch.load(load_loss_path, map_location = "cpu")
    return losses_dict["train"], losses_dict["val"]

In [ ]:
import Plotting.plotters_simple as plotters_simple

def make_plot_dataloader(dataset, stride = 1):

	"""
	Takes the training dataset and returns a dataloader of every 10th point
	to reduce visual clutter. 
	"""

	points_tensor, mass_matrices_tensor, input_matrices_tensor = dataset.tensors
	
	plot_sampled = points_tensor[::stride]
	mass_sampled = mass_matrices_tensor[::stride]
	input_sampled = input_matrices_tensor[::stride]

	plot_dataset = TensorDataset(plot_sampled, mass_sampled, input_sampled)
	plot_dataloader = DataLoader(plot_dataset, batch_size=len(plot_dataset), shuffle=False, num_workers=0)

	return(plot_dataloader)

In [ ]:
def plot_model_performance(model, plot_dataloader, folder_path):

	model_ana = autoencoders.Analytic_transformer(rp)

	os.makedirs(folder_path, exist_ok=True)

	model.eval()
	with torch.no_grad():
		for (q, M_q, A_q) in plot_dataloader:
			q = q.to(device)
			M_q = M_q.to(device)
			A_q = A_q.to(device)

			theta, J_h, q_hat, J_h_dec, J_h_ana = model(q)
			theta_ana = model.theta_ana(q)
			J_h_trans = torch.transpose(J_h, 1, 2)
			J_h_inv = J_h_dec
			J_h_inv_trans = torch.transpose(J_h_inv, 1, 2)

			J_h_inv_ana = torch.linalg.inv(J_h_ana)
			J_h_inv_trans_ana = torch.transpose(J_h_inv_ana, 1, 2)

			M_th = J_h_inv_trans @ M_q @ J_h_inv
			A_th = (J_h_inv_trans @ A_q).squeeze(-1)

			M_th_ana = J_h_inv_trans_ana @ M_q @ J_h_inv_ana
			A_th_ana = (J_h_inv_trans_ana @ A_q).squeeze(-1)


			A_th = A_th.cpu().detach().numpy()
			print("Percentage of abs(A_0) > 0.6:", 100 * np.sum(np.abs(A_th[:, 0]) > 0.6)/A_th[:, 0].size, "%")
			print("Percentage of abs(A_1) < 0.3:", 100 * np.sum(np.abs(A_th[:, 1]) < 0.3)/A_th[:, 1].size, "%")
			
			
			M_th_cpu = M_th.cpu().detach().numpy()
			print("Percentage of abs(M_00) > 1.0:", 100 * np.sum(np.abs(M_th_cpu[:, 0, 0]) > 1.0)/M_th_cpu[:, 0, 0].size, "%")
			print("Percentage of abs(M_01) < 0.2:", 100 * np.sum(np.abs(M_th_cpu[:, 0, 1]) < 0.2)/M_th_cpu[:, 0, 1].size, "%")
			print("Percentage of abs(M_11) > 1.0:", 100 * np.sum(np.abs(M_th_cpu[:, 1, 1]) > 1.0)/M_th_cpu[:, 1, 1].size, "%")

			plotting_3d = False
			plotting_2d = True
			if plotting_3d:
				plotters_simple.plot_3d_quad(q, [theta_ana[:, 0], theta[:, 0], theta_ana[:, 1], theta[:, 1]], "Analytic vs learned theta", 
											["th0_ana", "th0_learned", "th_1_ana", "th_1_learned"], "q_0", "q_1", "th", folder_path)
				plotters_simple.plot_3d_double(q, [A_th[:, 0], A_th[:, 1]], "Input decoupling", ["A0", "A1"], "q_0", "q_1", "A", folder_path)
				plotters_simple.plot_3d_quad(q, [M_th[:, 0, 0], M_th[:, 0, 1], M_th[:, 1, 0], M_th[:, 1, 1]], "M_th vs q", 
										 ["M_th[0,0]", "M_th[0,1]", "M_th[1,0]", "M_th[1,1]"], "q_0", "q_1", "M_th", folder_path)
				#plotters_simple.plot_3d_double(q, J_h_ana[:, 0, 0], J_h[:, 0, 1], "J_h", "00", "01", "q_0", "q_1", "J_h", device)
				#plotters_simple.plot_3d_double(q, J_h_ana[:, 1, 0], J_h[:, 1, 1], "J_h", "10", "11", "q_0", "q_1", "J_h", device)			
			
			if plotting_2d:
				plotters_simple.plot_2d_double(q, [A_th[:, 0], A_th[:, 1]], "Input matrix terms", [r"$A_{0}$", r"$A_{1}$"], r"$q_0$", r"$q_1$", r"$A$", folder_path)
				plotters_simple.plot_2d_quad(q, [M_th[:, 0, 0], M_th[:, 0, 1], M_th[:, 1, 0], M_th[:, 1, 1]], r"$M_{\theta}$" + " vs " + r"$q$", 
										 [r"$M_{\theta_{0,0}}$", r"$M_{\theta_{0,1}}$", r"$M_{\theta_{1,0}}$", r"$M_{\theta_{1,1}}$"], r"$q_{0}$", r"$q_{1}$", "M_th", folder_path)
				
			
w_diag         = 1. 
w_off_diag     = 1. 

w_input        = 1.
w_input_jac    = 1.
w_theta        = 1.
#f_inertia_eye  = 0. 

loss_weights = {
	"w_recon": w_recon,
	"w_J_eye": w_J_eye,
	"w_diag": w_diag,
	"w_off_diag": w_off_diag,
	"w_input": w_input,
	"w_input_jac": w_input_jac,
	"w_theta": w_theta
}

In [ ]:
q0_split = (-torch.pi, torch.pi)
batch_size = 512
train_part = 0.7

num_epochs = 3001
lr = 1e-3

# IMPORTANT:
# If you want to train on an existing model
#existing_model_name = "NN_202505131531"
existing_model_name = None

plt.ion()

current_time = datetime.now().strftime("%Y%m%d%H%M")

shifted_points = mask_points(q0_split, clockwise=train_clockwise)

dataset = make_dataset(shifted_points)
(train_dataloader, val_dataloader) = make_dataloaders(dataset=dataset, batch_size=batch_size, train_part=train_part)
outputs = []
model, train_losses, val_losses, dir_path, nn_filepath, file_counter = train_AE_model(rp, device, lr, num_epochs, q0_split, train_dataloader, 
														val_dataloader, current_time, loss_weights, existing_model_name, save_dir_name)

In [ ]:
save_model(model, nn_filepath)
save_loss_path = dir_path + "/losses.pt"
save_loss(train_losses, val_losses, save_loss_path)

In [ ]:
points_path = os.path.join(dir_path, "training_points_" + str(file_counter) + ".png")
points_plotter(shifted_points, save = True, file_path = points_path)
scatter_fig_path = os.path.join(dir_path, "loss_" + str(file_counter) + ".png")
plot_loss(train_losses, val_losses, log = True, save = True, file_path = scatter_fig_path)

In [ ]:
def theta_0_single(model, q):
	print("q:", q.size())
	theta = model.encoder(q)
	print("theta:", theta.size())
	return theta[:, 0].detach()
	
def theta_1_single(model, q):
	theta = model.encoder(q)
	return theta[:, 1].detach()
	
def q_hat_0_single(model, theta):
	q_hat = model.decoder(theta)
	return q_hat[:, 0].detach()
	
def q_hat_1_single(model, theta):
	q_hat = model.decoder(theta)
	return q_hat[:, 1].detach()

mapping_functions = (partial(theta_0_single,model), 
						partial(theta_1_single,model), 
						partial(q_hat_0_single,model), 
						partial(q_hat_1_single,model))
th_plotter = theta_visualizer.theta_plotter(rp=rp, n_lines=50, device=device, 
											mapping_functions=mapping_functions, mask_split=q0_split)
theta_figure_path = os.path.join(dir_path, "theta_mapping_" + str(file_counter) + ".png")
th_plotter.make_figure(theta_figure_path)
theta_anim_path = os.path.join(dir_path, "theta_mapping_" + str(file_counter) + ".mp4")
#th_plotter.make_animation(theta_anim_path, duration = 4, fps = 20)

#### Learned coordinate performance analysis

In [ ]:
test_points = mask_points(q0_split)
test_dataset = make_dataset(test_points)
test_dataloader = make_plot_dataloader(test_dataset, stride=10)

with torch.no_grad():
	for (q_test, M_q_test, A_q_test) in test_dataloader:
		q_test = q_test.to(device)
		M_q_test = M_q_test.to(device)
		A_q_test = A_q_test.to(device)

		theta_test, J_h_test, q_hat_test, J_h_dec_test, J_h_ana_test = model(q_test)
		theta_ana_test = model.theta_ana(q_test)
		J_h_trans_test = torch.transpose(J_h_test, 1, 2)
		J_h_inv_test = J_h_dec_test
		J_h_inv_trans_test = torch.transpose(J_h_inv_test, 1, 2)

		J_h_inv_ana_test = torch.linalg.inv(J_h_ana_test)
		J_h_inv_trans_ana_test = torch.transpose(J_h_inv_ana_test, 1, 2)

		M_th_test = J_h_inv_trans_test @ M_q_test @ J_h_inv_test
		A_th_test = (J_h_inv_trans_test @ A_q_test).squeeze(-1)

		M_th_ana_test = J_h_inv_trans_ana_test @ M_q_test @ J_h_inv_ana_test
		A_th_ana_test = (J_h_inv_trans_ana_test @ A_q_test).squeeze(-1)


In [ ]:
mse_A_th = F.mse_loss(A_th_test, A_th_ana_test)
diag_values_test = torch.diagonal(M_th_test, dim1=1, dim2=2)
mse_M_th_dia = torch.mean((-1 + torch.exp(-(diag_values_test - 1))) * (diag_values_test < 1).float())
mse_M_th_off_dia = F.mse_loss(M_th_test[:, 0, 1], torch.tensor(0.).repeat(M_th_test.size(0)).to(device))
mse_M_th = mse_M_th_dia + mse_M_th_off_dia
mse_recon = F.mse_loss(q_hat_test, q_test)
mse_J_eye = F.mse_loss(J_h_test @ J_h_dec_test, torch.eye(2).repeat(J_h_test.size(0), 1, 1).to(device))
print("MSE A_th  loss:", mse_A_th.item())
print("MSE M_th  loss:", mse_M_th.item())
print("MSE recon loss:", mse_recon.item())
print("MSE J_eye loss:", mse_J_eye.item())
mse_total = mse_A_th + mse_M_th + mse_recon + mse_J_eye
print("MSE total loss:", mse_total.item())

#### Learned coordinate performance plotting

In [ ]:
%matplotlib widget

folder_path = os.path.join(dir_path, "performance_plots_" + str(file_counter))

plt.ion()
plot_points = mask_points(q0_split, clockwise = train_clockwise)
plot_dataset = make_dataset(plot_points)
plot_dataloader = make_plot_dataloader(plot_dataset, stride = 1)
plot_model_performance(model, plot_dataloader, folder_path)


In [ ]:
from matplotlib.patches import Circle
import warnings
warnings.filterwarnings("ignore")


plt.rcParams.update({
	'font.size':       16,   # base font size for text
	'axes.titlesize':  14,   # title of each subplot
	'axes.labelsize':  14,   # x/y axis labels
	'xtick.labelsize': 12,   # x‐tick labels
	'ytick.labelsize': 12,   # y‐tick labels
	'legend.fontsize': 14,   # legend text (if you had one)
	'figure.titlesize': 18,  # suptitle
})


try:
	dir_name
except NameError:
	dir_name = "NN_202505141642(half-q)"

	dir_path = os.path.join(os.getcwd(), "Models", dir_name)
	nn_filename = "NN_202505141642" + "_0.pth"
	nn_filepath = os.path.join(dir_path, nn_filename)

	model_path = nn_filepath
	model = autoencoders.Autoencoder_double(rp).to(device)  # Initialize model architecture
	model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))  # Load weights


#"Models", dir_name, dir_name + "_" + str(file_counter-1) + ".pth")
model_ana = autoencoders.Analytic_transformer(rp)

models = [model_ana, model]
model_names = ["Analytic", "Learned"]
dir_path = dir_path
file_counter = 0


def check_clockwise_vectorized(q):
		"""
		Expects q to be a tensor of shape (N,2) where each row is [q0, q1].
		Returns two boolean masks: (cw_mask, ccw_mask), where:
			- cw_mask[i] is True if the i-th configuration is elbow clockwise.
			- ccw_mask[i] is True if the i-th configuration is elbow counterclockwise.
		
		The logic is as follows (from your original function):
			If q1 lies between q0 and q0+π, or between q0-2π and q0-π, then the configuration
			is considered counterclockwise. Otherwise it is clockwise.
		"""
		q0 = q[:, 0]
		q1 = q[:, 1]
		cond_ccw = ((q1 >= q0) & (q1 <= q0 + torch.pi))
		cw_mask = ~cond_ccw
		ccw_mask = cond_ccw
		return cw_mask, ccw_mask


# Define the number of grid points along each dimension.
n_points = 200

for i in range(2):

	if i == 0:
		# Create 1D tensors for q0 and q1 in the range [-pi, pi]
		q0_vals = torch.linspace(-np.pi, np.pi, n_points)
		q1_vals = torch.linspace(-np.pi, 2*np.pi, n_points)

		theta_xy_fig_path = os.path.join(dir_path, "theta_vs_q_full" + str(file_counter) + ".png")
	if i == 1:
		q0_vals = torch.linspace(training_data.q0_low, training_data.q0_high, n_points)
		q1_vals = torch.linspace(training_data.q1_low, training_data.q1_high, n_points)		
		theta_xy_fig_path = os.path.join(dir_path, "theta_vs_q_partial" + str(file_counter) + ".png")

	# Create a 2D grid (meshgrid) of q values.
	# (Note: using indexing='ij' so that the first axis corresponds to q0 and the second to q1)
	q0_grid, q1_grid = torch.meshgrid(q0_vals, q1_vals, indexing='ij')

	# Stack the grid to get a tensor of shape (n_points*n_points, 2)
	q_grid = torch.stack([q0_grid.flatten(), q1_grid.flatten()], dim=1).to(device)

	# === Compute theta0 and theta1 using the analytic encoder functions ===
	# We use torch.vmap to evaluate the functions over the batch of q values.
	# Note: encoder_theta_0_ana and encoder_theta_1_ana each return a tuple (theta, theta).

	if train_clockwise:
			raise ValueError("This plotter currently only supports plotting counterclockwise performance. Should be an easy fix")


	fig, axes = plt.subplots(2, 2, figsize=(10, 8.5))
	for i, (model, model_name) in enumerate(zip(models, model_names)):

			theta_out = model.encoder_vmap(q_grid)
			#theta_out = torch.vmap(model.encoder)(q_grid)

			theta0 = theta_out[:, 0]
			theta1 = theta_out[:, 1]

			# Since q1_grid and q2_grid are already on a mesh, we can compute x_end and y_end elementwise.
			x_end = rp["l0"] * torch.cos(q_grid[:, 0]) + rp["l1"] * torch.cos(q_grid[:, 1])
			y_end = rp["l0"] * torch.sin(q_grid[:, 0]) + rp["l1"] * torch.sin(q_grid[:, 1])

			# --- Determine configuration (clockwise vs. counterclockwise) for each q ---
			cw_mask, ccw_mask = check_clockwise_vectorized(q_grid)
			# Counterclockwise points
			x_end_ccw   = x_end[ccw_mask].detach().cpu().numpy()
			y_end_ccw   = y_end[ccw_mask].detach().cpu().numpy()
			theta0_ccw  = theta0[ccw_mask].detach().cpu().numpy()
			theta1_ccw  = theta1[ccw_mask].detach().cpu().numpy()
			thetas = [theta0_ccw, theta1_ccw]

			l_total = rp["l0"] + rp["l1"]

			for j in range(2):
					sc = axes[i, j].scatter(x_end_ccw, y_end_ccw, c=thetas[j], cmap='viridis', s=5)

					circle = Circle((0.0, 0.0),
									l_total*1.01,
									fill=False,
									linestyle='--',
									edgecolor='k',
									linewidth=1.0)
					axes[i, j].add_patch(circle)					
					axes[i, j].set_title(r"$\theta_{" + str(j) + "}$" + " - " + model_names[i])
					axes[i, j].set_xlabel("x")
					axes[i, j].set_ylabel("y")
					cbar = plt.colorbar(sc, ax=axes[i, j])
					cbar.set_label(r'$\theta$', fontsize=14)
					axes[i, j].set_xlim((-l_total*1.02, l_total*1.02))
					axes[i, j].set_ylim((-l_total*1.02, l_total*1.02))

	plt.tight_layout()
	plt.savefig(theta_xy_fig_path)
	plt.show()
